In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [ ]:
# create_pegmed()

In [ ]:
# create_pegprod()

In [ ]:
# create_prescriptions()

In [ ]:
# create_consultations()

In [ ]:
# create_clinicals()

In [ ]:
# create_tests()

In [ ]:
# create_referrals()

In [ ]:
# create_immunisations()

In [ ]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, female gender, insomnia) used in the logistic regression*

In [5]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [34]:
all_entries = get_all_entries()

In [ ]:
pt_features = create_pt_features()

In [ ]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [ ]:
# pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype) #does nothing if we're interested in all dementia

In [ ]:
pt_features = add_data_start_and_end_dates(all_entries,pt_features)

In [ ]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [33]:
all_entries

NameError: name 'all_entries' is not defined

In [11]:
for window in [sd.exposure_windows[1]]:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,req_yrs_post_index=sd.req_yrs_post_index,start_year=abs(window['start_year']))
    pt_features = delete_unmatched_cases_and_controls(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

10_to_5  being matched


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
len(pt_features)

22732

## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [21]:
medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [22]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [ ]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+sd.exposure_windows[1]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
create_pdd_for_each_drug(prescriptions,druglists.all_druglists,pt_features,sd.exposure_windows[1])

In [ ]:
# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in [sd.exposure_windows[1]]:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,codelists.all_codelists)
    pt_features,prescs,prescs_grouped,pt_pdds = create_PDD_columns_for_each_pt(pt_features,window,druglists.all_druglists,prescriptions)
#     pt_features = get_consultation_count(pt_features,all_entries,window)
    pt_features = create_quantiles_and_booleans(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)


10_to_5 ...


In [80]:
prescs_grouped

,patid,drug substance name,total_amount,benzo_and_z_drugs_100_pdds
0,4289,Diazepam,1100.0,2.200000
1,7301,Diazepam,236.0,0.472000
2,7301,Temazepam,840.0,0.600000
3,7301,Zopiclone,315.0,0.525000
4,7330,Clobazam,300.0,0.272727
5,22024,Diazepam,40.0,0.080000
6,24081,Diazepam,42.0,0.084000
7,24081,Temazepam,140.0,0.100000
8,26002,Zolpidem tartrate,560.0,0.700000
9,37099,Temazepam,840.0,0.600000


In [81]:
pt_pdds

,patid,benzo_and_z_drugs_100_pdds
0,4289,2.200000
1,7301,1.597000
2,7330,0.272727
3,22024,0.080000
4,24081,0.184000
5,26002,0.700000
6,37099,0.600000
7,40131,11.613333
8,45315,0.280000
9,47113,0.120000


In [55]:
len(pt_features)

22732

In [56]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'insomnia', 'stroke',
       'intellectual_disability', 'non_stroke_vascular_disease',
       'hypertension', 'diabetes', 'mental_illness_non_smi',
       'mental_illness_smi', 'sleep_apnoea', 'chronic_pulmonary_disease',
       'epilepsy', 'non_insomnia_GP_consultations', 'benzo_and_z_drugs_any',
       'insomnia_any', 'insomnia_count:0', 'insomnia_count:1_5',
       'insomnia_count:6_10', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:abov

In [60]:
new_cols = [col for col in pt_features.columns if '_pdds:' not in col]

In [61]:
pt_features = pt_features[new_cols]

In [62]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ sd.exposure_windows[1]['name'] +'.csv',index=False)


In [66]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'insomnia', 'stroke',
       'intellectual_disability', 'non_stroke_vascular_disease',
       'hypertension', 'diabetes', 'mental_illness_non_smi',
       'mental_illness_smi', 'sleep_apnoea', 'chronic_pulmonary_disease',
       'epilepsy', 'non_insomnia_GP_consultations', 'benzo_and_z_drugs_any',
       'insomnia_any', 'insomnia_count:0', 'insomnia_count:1_5',
       'insomnia_count:6_10', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:abov